# Final project

Philip Wollsén Ervius \
phao21@student.bth.se

Amin Afzali \
moaf@student.bth.se

### Libraries

In [5]:
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_b7
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, recall_score, f1_score

from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline


### Preparations

We will be using a pretrained Residual Neural Network called ResNet50. Here's how we get it.

In [ ]:
# Preprocessing for CIFAR-100 (resize and normalize to match ImageNet-1K pretraining)

preprocess = transforms.Compose([
    transforms.Resize(256),       # Resize to 256px on the shorter side
    transforms.CenterCrop(224),   # Center crop to 224x224
    transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    ),
])

### Download the CIFAR-100 dataset

In [ ]:
# Load CIFAR-100 dataset for evaluation
dataset = CIFAR100(root="./data", train=False,
                   transform=preprocess, download=True)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False, num_workers=4)

### Load the pretrained model

In [ ]:
model = efficientnet_b7()
model.eval()

### Define a function to evaluate the model

In [ ]:

# Evaluate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

criterion = CrossEntropyLoss()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        accuracy(outputs, labels)
        f1_score(outputs, labels)
        recall(outputs, labels)


print(f"Test Loss: {test_loss / len(test_loader):.4f}")
print(f"Test Accuracy: {accuracy:.2f}%")


## The first attack